In [13]:
from apposto.utils.zernike_generator import ZernikeGenerator
import numpy as np

In [14]:
def reconstructor_1(ima, zernike_modes):
    zg = ZernikeGenerator(ima.shape[0])
    n_punti = ima.compressed().shape[0]
    interaction_mat = np.zeros((n_punti, zernike_modes.size))
    for i in range(zernike_modes.size):
        z = zg.getZernike(zernike_modes[i])
        z_new = np.ma.masked_array(z, mask=ima.mask)
        interaction_mat.T[i] = z_new.compressed()

    reconstructor = np.linalg.pinv(interaction_mat)
    coeff = np.dot(reconstructor, ima.compressed())
    return interaction_mat, reconstructor, coeff

def reconstructor_2(ima, zernike_modes):
    zg = ZernikeGenerator(ima.shape[0])
    n_punti = ima.compressed().shape[0]
    int_mat = np.zeros((zernike_modes.size, n_punti))
    for i in range(zernike_modes.size):
        z = zg.getZernike(zernike_modes[i])
        z_new = np.ma.masked_array(z, mask=ima.mask)
        int_mat[i] = z_new.compressed()
    rec = np.linalg.pinv(int_mat)
    coeff = np.dot(ima.compressed(), rec)
    return int_mat, rec, coeff

In [15]:
def createCircularMask(ima_shape, mask_center, mask_radius):
    Y, X = np.indices(ima_shape)
    dist_from_center = np.sqrt((X - mask_center[0])**2 + (Y - mask_center[1])**2)
    mask = dist_from_center <= mask_radius
    return mask

def getFootprintImage(full_ima, fp_center, fp_radius):
    mask = createCircularMask(full_ima.shape, fp_center, fp_radius)
    fp_ima = np.ma.masked_where(mask == False, full_ima)
    return fp_ima

In [16]:
mp_diameter_in_pixel = 1024
fp_diameter_in_pixel = 200
fp_center = (450,500)

zg_mp = ZernikeGenerator(mp_diameter_in_pixel)
ima_mp = zg_mp.getZernike(2) + zg_mp.getZernike(3) + zg_mp.getZernike(4)
ima_fp = getFootprintImage(ima_mp, fp_center, fp_diameter_in_pixel/2)

In [17]:
zern_array = np.array([2, 3, 4])
interaction_mp, reconstructor_mp, a_mp = reconstructor_1(ima_mp, zern_array)
interaction_fp, reconstructor_fp, a_fp = reconstructor_1(ima_fp, zern_array)

In [27]:
print('Shape of MP interaction matrix: {} \n'.format(interaction_mp.shape))
print('Shape of FP interaction matrix: {} \n'.format(interaction_fp.shape))
print('Shape of MP reconstructor: {} \n'.format(reconstructor_mp.shape))
print('Shape of FP reconstructor: {} \n'.format(reconstructor_fp.shape))
print('MP coefficients: {} \n'.format(a_mp))
print('FP coefficients: {} \n'.format(a_fp))

Shape of MP interaction matrix: (823592, 3) 

Shape of FP interaction matrix: (31417, 3) 

Shape of MP reconstructor: (3, 823592) 

Shape of FP reconstructor: (3, 31417) 

MP coefficients: [1. 1. 1.] 

FP coefficients: [1. 1. 1.] 

